# Data Fetch

### Data fetch and pre process from 2012-2018 for training and from 2019-2020 for testing 

In [1]:
from pandas import DataFrame, read_csv
from bs4 import BeautifulSoup
import requests
import re
from time import sleep
from random import randrange
from math import sqrt

### Reading pre career data from 2008 till 2011

In [2]:
daf=read_csv("pre-2008-2011.csv", encoding = "latin")

In [3]:
player_name=list(daf["name"]) #pl_name
played=list(daf["played"]) #pl_played

#batting order
run_bat=list(daf["run_bat"]) #bat_run
f_50s=list(daf["50s"])  #fi_50s
h_100s=list(daf["100s"]) #hu_100s
batted=list(daf["batted"]) #bat_batted
out=list(daf["out"]) #bat_out

#bowling order
bowled=list(daf["bowled"]) #boll_bowled
wkt_tacken=list(daf["wkt_tacken"]) #boll_wkt
wkt_5=list(daf["5_wkt_taken"]) #boll_5
run_given=list(daf["run_given_bowler"]) #boll_run
econ=list(daf["econ"]) #boll_eco
over=list(daf["over"]) #boll_ove

In [4]:
#final
toss_win=[] #teamA 1 /teamB 0 
team_strength_val=[] #value
winner_match=[] # teamA 1/ teamB 0 #target
playground=[] #teamA 0/ teamB 1/ neutral 2

# Batsman strength calculation

In [5]:
class bat:
    def __init__(self, pl_name, player, played, batted, hu_100s, fi_50s, run, out):
        self.pl_name=pl_name
        self.played=played
        self.batted=batted
        self.hu_100s=hu_100s
        self.fi_50s=fi_50s
        self.run=run
        self.out=out
        self.player=player
        self.career=[]
        self.recent=[]
        self.batsman_score=[]
    def model_bat(self,last):
        """
        year1=read_csv(str(self.year-4)+".csv", encoding = "latin")
        year2=read_csv(str(self.year-3)+".csv", encoding = "latin")
        year3=read_csv(str(self.year-2)+".csv", encoding = "latin")
        year4=read_csv(str(self.year-1)+".csv", encoding = "latin")
        """
        for pl in self.pl_name:
            ind=self.player.index(pl)
            last1=last
            u=float(self.batted[ind]/self.played[ind])
            u=sqrt(u)
            v=float((20*self.hu_100s[ind])+(5*self.fi_50s[ind]))
            bat_avg=""
            if self.out[ind]!=0:
                bat_avg=float(self.run[ind]/self.out[ind])
            else:
                bat_avg=self.run[ind]
            w=float((0.3*v)+(0.7*bat_avg))
            
            #career_score
            self.career.append(float(w*u))
            
            #last four match
            #recent_score calculation
            total=0
            count=0
            while last1 and count!=4:
                file=read_csv("C:/Users/HP/winnning/matches/"+str(last1)+".csv", encoding = "latin")
                val=file[file["name"]==pl].index.values
                if val:
                    total+=file["bat_run"][val[0]]
                    count+=1
                last1-=1
            """
            val=year2[year2["name"]==pl].index.values
            if val:
                total+=year2["run_bat"][val[0]]
            val=year3[year3["name"]==pl].index.values
            if val:
                total+=year3["run_bat"][val[0]]
            val=year4[year4["name"]==pl].index.values
            if val:
                total+=year4["run_bat"][val[0]]
            """
            if count==4:
                self.recent.append(float(total/4))
            elif count!=0:
                self.recent.append(float(total/count))
            elif count==0:
                self.recent.append(float(total))
        #batsman_score calculate    
        for i in range(len(self.pl_name)):
            #index=self.pl_name.index(pl)
            self.career[i]=float(self.career[i]/max(self.career))
            self.recent[i]=float(self.recent[i]/max(self.recent))
            self.batsman_score.append(round(float(0.35*self.career[i]+0.65*self.recent[i]),5))
        return self.batsman_score


# Bowling strength calculation

In [6]:
class ball:
    def __init__(self, pl_name, player, played, bowled, wkt, wkt_5, run, over):
        self.pl_name=pl_name
        self.player=player
        self.played=played
        self.bowled=bowled
        self.wkt=wkt
        self.wkt_5=wkt_5
        self.run=run
        self.over=over
        self.ball_score=[]
    def model_ball(self):
        for pl in self.pl_name:
            index=self.player.index(pl)
            u=float(self.bowled[index]/self.played[index])
            u=sqrt(u)
            v=float((10*self.wkt_5[index])+(5*self.wkt[index]))
            ball_avg=""
            if self.wkt[index]!=0:
                ball_avg=float(self.run[index]/self.wkt[index])
            else:
                ball_avg=float(self.run[index])
            econ=""
            if self.over[index]!=0:
                econ=float(self.run[index]/self.over[index])
            else:
                econ=0
            w=float(ball_avg*econ)
            if w!=0:
                self.ball_score.append(round(float((u*v)/w),5))
            else:
                self.ball_score.append(round(float(u*v),5))
        return self.ball_score

# Team Strength Calculation

In [7]:
class team_strength:
    def __init__(self, pl_name, ball_score, batsman_score, team):
        self.ball_score=ball_score
        self.batsman_score=batsman_score
        self.pl_name=pl_name
        self.team=team
    def strength(self):
        bat_strongA=0
        bat_strongB=0
        ball_strongA=0
        ball_strongB=0

        for pl in self.pl_name:
            index=self.pl_name.index(pl)
            self.batsman_score[index]=float(self.batsman_score[index]/max(self.batsman_score))
            self.ball_score[index]=float(self.ball_score[index]/max(self.ball_score))
        for pl in self.pl_name:
            index=self.pl_name.index(pl)
            if self.team[index]=="teamA":
                bat_strongA+=float(self.batsman_score[index])
                ball_strongA+=float(self.ball_score[index])
            elif self.team[index]=="teamB":
                bat_strongB+=float(self.batsman_score[index])
                ball_strongB+=float(self.ball_score[index])
        relative_strength=float((bat_strongA/ball_strongB)-(bat_strongB/ball_strongA))
        return [round(bat_strongA,5),round(ball_strongA,5), round(bat_strongB,5),
                round(ball_strongB,5), round(relative_strength,5)]   

# Data fetch from 2012 till 2018 and store in final train csv file

In [13]:
total=0
match_count=616
ground_csv=read_csv("ground.csv", encoding = "latin")
ground_city=list(ground_csv["City"])
ground_country=list(ground_csv["Country"])
for i in range(1, 5):
    newUrl="https://stats.espncricinfo.com/ci/engine/stats/index.html?ballsmax1=300;ballsval1=balls;class=2;filter=advanced;floodlit=1;floodlit=2;home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;opposition=1;opposition=2;opposition=25;opposition=3;opposition=4;opposition=5;opposition=6;opposition=7;opposition=8;orderby=start;page="+str(i)+";result=1;spanmax2=31+Dec+2020;spanmin2=01+Jan+2018;spanval2=span;team=1;team=2;team=25;team=3;team=4;team=5;team=6;team=7;team=8;template=results;type=team;view=results"
    #print(newUrl)
    response=requests.get(newUrl,timeout=60)
    soup=BeautifulSoup(response.text, 'lxml')
    table = soup.find_all("div",attrs={"class": "engine-dd"})
    res=[]
    for result in table:
        s=result.findAll('a', attrs={'href': re.compile("^/ci/engine/match/")})
        if s:
            href=s[0].get('href')
            res.append("https://stats.espncricinfo.com/"+href)
            
    #bolling and batting calculation
    #batting
    #venue
    table = soup.find_all("div",attrs={"class": "engine-dd"})
    table = soup.find_all('table')[2] 
    table_body=table.find('tbody')
    rows = table_body.find_all('tr')
    winner=[]
    venue_g=[]
    toss=[]
    for row in rows[:]:
        cols=row.find_all('td')
        colm=[]
        for x in cols:
            colm.append(x.text.strip().lower())
        winner.append(colm[0])
        if colm[4]=="won":
            toss.append(colm[0])
        else:
            con=colm[7].split("v ")[1].strip()
            toss.append(con)
        venue_g.append(colm[8].split("(")[0].strip().lower())
    print("venue len",len(venue_g))
    print(res[0])
    pointer=0
    for incre in range(len(res)):
        url=res[incre]
        print(url)
        pl_name=[]
        #boll
        pl_played=[]
        boll_eco=[]
        boll_ove=[]
        boll_bowled=[]
        boll_wkt=[]
        boll_5=[]
        boll_run=[]
        #batsman
        bat_run=[]
        fi_50s=[]
        hu_100s=[]
        bat_batted=[]
        bat_out=[]
        not_batted=[]
        team_belong=[]
        try:
            res_response=requests.get(url, timeout=60)
            res_soup=BeautifulSoup(res_response.text,'lxml')

            #team data collected
            team=res_soup.find_all("div", attrs={"class":"cscore_link cscore_link--button"})
            team_name=team[1].find_all("span", attrs={"class":"cscore_name cscore_name--long"})
            teamA=team_name[0].text.strip().lower() #0
            teamB=team_name[1].text.strip().lower() #1

            """
            #Venue / Ground
            venue=soup.find("article", attrs={"class":"sub-module game-information pre"})
            venue=venue.find("div", attrs={"class":"stadium-details"})
            venue=venue.find("span")
            venue=venue.text.split(",")[1].strip()
            """
            try:
                print(incre)
                print(venue_g[incre],"----venue---",incre)
                #ground_csv=read_csv("ground.csv", encoding = "latin")
                #ground=ground_csv[ground_csv["City"]==venue_g[incre]].index.values
                #venue=ground_csv["Country"][ground]
                ground_venue=""
                if venue_g[incre] in ground_city:
                    ind=ground_city.index(venue_g[incre])
                    ground_venue=ground_country[ind]
                    print("index of ground", ind)
                #print("venue","============",venue,"-----venue city----", venue_g[incre])
                #toss
                tos=toss[incre]
                win=winner[incre]
                pointer+=1

                table = res_soup.find_all("article",attrs={"class": "sub-module scorecard"})

                #score of one row
                for score_section in table:
                    batting_team=score_section.find("h2").text.strip().lower()
                    print("bat",batting_team, teamA, teamB)
                    if teamA in batting_team:
                        print("batting team teamA--->", teamA)
                    elif teamB in batting_team:
                        print("batting team teamB--->",teamB)
                    score_b=score_section.find_all("div", attrs={"class":"wrap batsmen"})
                    for result in score_b:
                        #name=result.find("a").text
                        comm=result.find("div", attrs={"class":"cell commentary"})
                        run=result.findAll("div", attrs={"class":"cell runs"})
                        batRun=[]
                        for i in run:
                            if i:
                                batRun.append(i.text)
                        #print(name.text,"--------",comm.text)
                        #how many out
                        bats=result.find("div", attrs={"class":"cell batsmen"})
                        bats=bats.text.split("†")[0].strip().split("(c)")[0].strip().lower()
                        comm=result.find("div", attrs={"class":"cell commentary"})
                        if bats in pl_name:
                            pl_index=pl_name.index(bats)
                            if comm.text!="not out":
                                bat_out[pl_index]+=1
                            if int(batRun[0])>=100:
                                hu_100s[pl_index]+=1
                            elif int(batRun[0])>=50:
                                fi_50s[pl_index]+=1
                            if bat_batted[pl_index]==0:
                                bat_batted[pl_index]=1
                            bat_run[pl_index]+=int(batRun[0])
                        else:
                            try:
                                bat_run.append(int(batRun[0]))
                                pl_name.append(bats)
                                if comm.text!="not out":
                                    bat_out.append(1)
                                else:
                                    bat_out.append(0)

                                bat_batted.append(1)
                                if int(batRun[0])>=100:
                                    hu_100s.append(1)
                                    fi_50s.append(0)
                                elif int(batRun[0])>=50:
                                    fi_50s.append(1)
                                    hu_100s.append(0)
                                else:
                                    hu_100s.append(0)
                                    fi_50s.append(0)
                                boll_ove.append(0)
                                boll_bowled.append(0)
                                boll_wkt.append(0)
                                boll_5.append(0)
                                boll_run.append(0)
                                boll_eco.append(0)
                                pl_played.append(1)
                                if teamA in batting_team:
                                    team_belong.append("teamA")
                                elif teamB in batting_team:
                                    team_belong.append("teamB")
                            except:
                                    print("error", batRun[0], "type",type(batRun[0]))
                                    pass
                    #bowling score
                    score_bal=score_section.find("div", attrs={"class":"scorecard-section bowling"})
                    ball_table=score_bal.find("table")
                    body=ball_table.find("tbody")
                    rows=body.find_all("tr")
                    for row in rows:
                            cols=row.find_all("td")
                            colm=[]
                            for x in cols:
                                colm.append(x.text.strip())
                            colmN=colm[0].split("†")[0].strip().split("(c)")[0].strip().lower()
                            if colmN in pl_name:

                                pl_index=pl_name.index(colmN)
                                boll_ove[pl_index]+=float(colm[2])  #2
                                boll_run[pl_index]+=int(colm[4]) #4
                                boll_wkt[pl_index]+=int(colm[5]) #5
                                if boll_eco[pl_index]:
                                    boll_eco[pl_index]=(boll_eco[pl_index]+float(colm[6]))/2 #6
                                else:
                                    boll_eco[pl_index]=float(colm[6])
                                if  int(colm[5])>=5:
                                    boll_5[pl_index]+=1
                                if boll_bowled[pl_index]==0:
                                    boll_bowled[pl_index]=1

                            else:
                                colmN=colm[0].split("†")[0].strip().split("(c)")[0].strip().lower()
                                pl_name.append(colmN) #0
                                boll_ove.append(float(colm[2]))  #2
                                boll_run.append(int(colm[4])) #4
                                boll_wkt.append(int(colm[5])) #5
                                boll_eco.append(float(colm[6])) #6
                                hu_100s.append(0)
                                fi_50s.append(0)
                                bat_batted.append(0)
                                bat_run.append(0)
                                bat_out.append(0)
                                if  int(colm[5])>=5:
                                    boll_5.append(1)
                                else:
                                    boll_5.append(0)
                                boll_bowled.append(1)
                                pl_played.append(1)
                                if teamA in batting_team:
                                    team_belong.append("teamB")
                                elif teamB in batting_team:
                                    team_belong.append("teamA")

                    # not batted player
                    not_bat=score_section.find("div", attrs={"class":"wrap dnb"})
                    cell=not_bat.find("div", attrs={"class":"cell"})
                    row=cell.find_all("a")
                    if row:
                        for rows in row:
                                play_not=rows.text.replace(',',"")
                                play_not=play_not.strip()
                                play_not=play_not.split("†")[0].strip().split("(c)")[0].strip().lower()
                                if play_not not in pl_name:
                                        pl_name.append(play_not)
                                        boll_ove.append(0)  #2
                                        boll_run.append(0) #4
                                        boll_wkt.append(0) #5
                                        boll_eco.append(0) #6
                                        boll_5.append(0)
                                        boll_bowled.append(0)
                                        hu_100s.append(0)
                                        fi_50s.append(0)
                                        bat_batted.append(0)
                                        bat_run.append(0)
                                        bat_out.append(0)
                                        pl_played.append(1)
                                        if teamA in batting_team:
                                            team_belong.append("teamA")
                                        elif teamB in batting_team:
                                            team_belong.append("teamB")    

                #final data updation before moving to next match

                for pl in pl_name:
                    player_id=pl_name.index(pl)
                    if pl in player_name:
                        ind=player_name.index(pl)
                        #name[index] #pl_name
                        played[ind]+=pl_played[player_id]

                        #batting order
                        run_bat[ind]+=bat_run[player_id]
                        f_50s[ind]+=fi_50s[player_id]
                        h_100s[ind]+=hu_100s[player_id]
                        batted[ind]+=bat_batted[player_id]
                        out[ind]+=bat_out[player_id]

                        #bowling order
                        bowled[ind]+=boll_bowled[player_id]
                        wkt_tacken[ind]+=boll_wkt[player_id]
                        wkt_5[ind]+=boll_5[player_id]
                        run_given[ind]+=boll_run[player_id]
                        econ[ind]=(econ[ind]+boll_eco[player_id])/2
                        over[ind]+=boll_ove[player_id]
                    else:
                        player_name.append(pl)
                        played.append(pl_played[player_id]) #pl_played

                        #batting order
                        run_bat.append(bat_run[player_id])
                        f_50s.append(fi_50s[player_id])
                        h_100s.append(hu_100s[player_id])
                        batted.append(bat_batted[player_id])
                        out.append(bat_out[player_id])

                        #bowling order
                        bowled.append(boll_bowled[player_id])
                        wkt_tacken.append(boll_wkt[player_id])
                        wkt_5.append(boll_5[player_id])
                        run_given.append(boll_run[player_id])
                        econ.append(boll_eco[player_id])
                        over.append(boll_ove[player_id])
                batt=bat(pl_name, player_name, played, batted, h_100s, f_50s, run_bat, out)
                batt=batt.model_bat(match_count)
                match_count+=1
                baall=ball(pl_name, player_name, played, bowled, wkt_tacken, wkt_5, run_given, over)
                baall=baall.model_ball()
                team_stren=team_strength(pl_name, baall, batt, team_belong)
                team_stren=team_stren.strength()
                print(team_stren)
                
                dft=DataFrame()
                dft["name"]=pl_name #0
                dft["boll_bowled"]=boll_bowled
                dft["over"]=boll_ove  #2
                dft["boll_run"]=boll_run #4
                dft["wkt"]=boll_wkt #5
                dft["econ"]=boll_eco #6
                dft["5_wkt"]=boll_5
                dft["bat_run"]=bat_run
                dft["bat_out"]=bat_out
                dft["bat_batted"]=bat_batted
                dft["pl_paly"]=pl_played
                dft["fifties"]=fi_50s
                dft["centuries"]=hu_100s
                dft["team_belong"]=team_belong
                dft.to_csv("C:/Users/HP/winnning/matches/"+str(match_count)+".csv", mode="a",header=True)
                #dft["batsman_score"]=batt
                #dft["ball_score"]=baall
                #print(len(team_belong), len(pl_name), len(baall))
                print(dft.head(22))
                
                print("tos", tos)
                if win==teamA:
                    winner_match.append(1)
                else:
                    winner_match.append(0)
                if tos==teamA:
                    print("teamA", teamA,"winner",win)
                    toss_win.append(1)
                elif tos==teamB:
                    toss_win.append(0)
                    print("teamB", teamB,"winner", win)
                if ground_venue==teamA:
                    playground.append(1)
                elif ground_venue==teamB:
                    playground.append(0)
                else:
                    playground.append(2)
                team_strength_val.append(team_stren)
                print("ground_venue-----------",ground_venue,"toss", tos)



            except:
                print(incre)
                pointer+=1
                print("error: ground")
                print(len(venue_g))
        except:
            print("error: url problem")
    #total+=len(res)
    ran=randrange(0,4,2)
    sleep(ran)


venue len 50
https://stats.espncricinfo.com//ci/engine/match/1115802.html
https://stats.espncricinfo.com//ci/engine/match/1115802.html
0
wellington ----venue--- 0
index of ground 10
bat new zealand innings new zealand pakistan
batting team teamA---> new zealand
bat pakistan innings new zealand pakistan
batting team teamB---> pakistan
[4.31713, 2.48462, 5.56164, 2.6966, -0.63747]
               name  boll_bowled  over  boll_run  wkt  econ  5_wkt  bat_run  \
0        mj guptill            0   0.0         0    0  0.00      0       48   
1           c munro            0   0.0         0    0  0.00      0       58   
2     ks williamson            0   0.0         0    0  0.00      0      115   
3       lrpl taylor            0   0.0         0    0  0.00      0       12   
4        twm latham            0   0.0         0    0  0.00      0        3   
5       hm nicholls            0   0.0         0    0  0.00      0       50   
6        mj santner            1   6.0        39    0  6.50      

3
melbourne ----venue--- 3
index of ground 0
bat australia innings australia england
batting team teamA---> australia
bat england innings australia england
batting team teamB---> england
[2.73446, 2.51322, 4.14757, 3.87242, -0.94416]
           name  boll_bowled  over  boll_run  wkt  econ  5_wkt  bat_run  \
0      aj finch            0   0.0         0    0   0.0      0      107   
1     da warner            0   0.0         0    0   0.0      0        2   
2     spd smith            0   0.0         0    0   0.0      0       23   
3       tm head            1   0.5         6    0   7.2      0        5   
4      mr marsh            1   2.0        15    0   7.5      0       50   
5    mp stoinis            1   6.0        33    1   5.5      0       60   
6      td paine            0   0.0         0    0   0.0      0       27   
7    pj cummins            1  10.0        63    2   6.3      0       12   
8      ma starc            1  10.0        71    2   7.1      0        0   
9        aj tye 

6
brisbane ----venue--- 6
index of ground 22
bat australia innings australia england
batting team teamA---> australia
bat england innings australia england
batting team teamB---> england
[3.20704, 1.61589, 4.68776, 3.40763, -1.9599]
             name  boll_bowled  over  boll_run  wkt  econ  5_wkt  bat_run  \
0       da warner            0   0.0         0    0  0.00      0       35   
1        aj finch            1   3.0        17    0  5.66      0      106   
2       spd smith            0   0.0         0    0  0.00      0       18   
3         tm head            1   7.0        55    0  7.85      0        7   
4        mr marsh            1   1.0         4    0  4.00      0       36   
5      mp stoinis            1   4.2        34    0  7.84      0        4   
6        cl white            0   0.0         0    0  0.00      0       15   
7        at carey            0   0.0         0    0  0.00      0       27   
8        ma starc            1  10.0        59    4  5.90      0        3 

9
dhaka ----venue--- 9
index of ground 84
bat bangladesh innings bangladesh sri lanka
batting team teamA---> bangladesh
bat sri lanka innings bangladesh sri lanka
batting team teamB---> sri lanka
[3.97876, 3.2193, 5.58748, 2.90328, -0.36518]
                 name  boll_bowled  over  boll_run  wkt   econ  5_wkt  \
0         tamim iqbal            0   0.0         0    0   0.00      0   
1        anamul haque            0   0.0         0    0   0.00      0   
2     shakib al hasan            1   1.0        10    0  10.00      0   
3     mushfiqur rahim            0   0.0         0    0   0.00      0   
4         mahmudullah            0   0.0         0    0   0.00      0   
5       sabbir rahman            0   0.0         0    0   0.00      0   
6          abul hasan            1   4.0        25    0   6.25      0   
7       nasir hossain            1   3.0        19    0   6.33      0   
8    mashrafe mortaza            1   2.0        15    0   7.50      0   
9       rubel hossain       

13
durban ----venue--- 13
index of ground 104
bat south africa innings south africa india
batting team teamA---> south africa
bat india innings south africa india
batting team teamB---> india
[2.95222, 2.85563, 4.35519, 2.4443, -0.31733]
              name  boll_bowled  over  boll_run  wkt   econ  5_wkt  bat_run  \
0        q de kock            0   0.0         0    0   0.00      0       34   
1          hm amla            0   0.0         0    0   0.00      0       16   
2     f du plessis            0   0.0         0    0   0.00      0      120   
3       ak markram            1   2.0        20    0  10.00      0        9   
4        jp duminy            1   2.0        16    0   8.00      0       12   
5        da miller            0   0.0         0    0   0.00      0        7   
6        ch morris            1   7.0        52    0   7.42      0       37   
7   al phehlukwayo            1   8.0        42    2   5.25      0       27   
8         k rabada            1   9.3        48    

...

1
mumbai ----venue--- 1
index of ground 75
bat india innings india australia
batting team teamA---> india
bat australia innings india australia
batting team teamB---> australia
[3.10268, 2.89538, 4.58118, 4.61356, -0.90973]
              name  boll_bowled  over  boll_run  wkt  econ  5_wkt  bat_run  \
0        rg sharma            0   0.0         0    0  0.00      0       10   
1         s dhawan            0   0.0         0    0  0.00      0       74   
2         kl rahul            0   0.0         0    0  0.00      0       47   
3          v kohli            0   0.0         0    0  0.00      0       16   
4          ss iyer            0   0.0         0    0  0.00      0        4   
5          rr pant            0   0.0         0    0  0.00      0       28   
6        ra jadeja            1   8.0        41    0  5.12      0       25   
7        sn thakur            1   5.0        43    0  8.60      0       13   
8   mohammed shami            1   7.4        58    0  7.56      0       10

4
cape town ----venue--- 4
index of ground 99
bat england innings england south africa
batting team teamA---> england
bat south africa innings england south africa
batting team teamB---> south africa
[3.79538, 2.09309, 3.56607, 2.84151, -0.36804]
                 name  boll_bowled  over  boll_run  wkt  econ  5_wkt  bat_run  \
0              jj roy            0   0.0         0    0  0.00      0       32   
1         jm bairstow            0   0.0         0    0  0.00      0       19   
2             je root            1   7.0        35    1  5.00      0       17   
3          ejg morgan            0   0.0         0    0  0.00      0       11   
4            jl denly            1   6.0        41    0  6.83      0       87   
5            t banton            0   0.0         0    0  0.00      0       18   
6           sm curran            1   5.0        28    0  5.60      0        7   
7           cr woakes            1   9.0        36    1  4.00      0       40   
8           tk curran   

7
johannesburg ----venue--- 7
index of ground 102
bat south africa innings south africa england
batting team teamA---> south africa
bat england innings south africa england
batting team teamB---> england
[2.64369, 2.2049, 5.75863, 1.91519, -1.23137]
                 name  boll_bowled  over  boll_run  wkt   econ  5_wkt  \
0           q de kock            0   0.0         0    0   0.00      0   
1        rr hendricks            0   0.0         0    0   0.00      0   
2            t bavuma            0   0.0         0    0   0.00      0   
3   he van der dussen            0   0.0         0    0   0.00      0   
4            jt smuts            1   3.0        12    0   4.00      0   
5           da miller            0   0.0         0    0   0.00      0   
6      al phehlukwayo            1   2.0        23    0  11.50      0   
7        be hendricks            1  10.0        59    3   5.90      0   
8           l sipamla            1   9.2        42    1   4.50      0   
9           tk curra

10
hambantota ----venue--- 10
index of ground 177
bat sri lanka innings sri lanka west indies
batting team teamA---> sri lanka
bat west indies innings sri lanka west indies
batting team teamB---> west indies
[4.11367, 3.32928, 4.20207, 3.35054, -0.03439]
               name  boll_bowled  over  boll_run  wkt  econ  5_wkt  bat_run  \
0      wia fernando            0   0.0         0    0  0.00      0      127   
1   fdm karunaratne            0   0.0         0    0  0.00      0        1   
2       mdkj perera            0   0.0         0    0  0.00      0        0   
3        bkg mendis            0   0.0         0    0  0.00      0      119   
4       nltc perera            1   5.0        18    0  3.60      0       36   
5        ad mathews            1   5.0        20    1  4.00      0        1   
6       dm de silva            0   0.0         0    0  0.00      0       12   
7      pwh de silva            1  10.0        30    3  3.00      0       17   
8           i udana            1  

13
bloemfontein ----venue--- 13
index of ground 103
bat australia innings australia south africa
batting team teamA---> australia
bat south africa innings australia south africa
batting team teamB---> south africa
[4.5171, 2.24625, 4.26762, 3.1971, -0.48701]
              name  boll_bowled  over  boll_run  wkt   econ  5_wkt  bat_run  \
0        da warner            0   0.0         0    0   0.00      0       35   
1         aj finch            0   0.0         0    0   0.00      0       69   
2        spd smith            0   0.0         0    0   0.00      0       13   
3    m labuschagne            1   1.0        11    0  11.00      0        0   
4        djm short            0   0.0         0    0   0.00      0       69   
5         mr marsh            1   9.0        51    0   5.66      0       36   
6         at carey            0   0.0         0    0   0.00      0       21   
7          ac agar            1   9.0        48    0   5.33      0        9   
8       pj cummins            

#### DataFrame for final data having strength, ground value, toss, winner of match

In [14]:
ndff=DataFrame()
ndff["winner"]=winner_match
ndff["toss"]=toss_win
ndff["ground"]=playground
strength=[]
bat_strengthA=[]
ball_strengthA=[]
bat_strengthB=[]
ball_strengthB=[]
for val in team_strength_val:
    strength.append(val[4])
    bat_strengthA.append(val[0])
    ball_strengthA.append(val[1])
    bat_strengthB.append(val[2])
    ball_strengthB.append(val[3])
ndff["strength"]=strength
ndff["bat_strengthA"]=bat_strengthA
ndff["ball_strengthA"]=ball_strengthA
ndff["bat_strengthB"]=bat_strengthB
ndff["ball_strengthB"]=ball_strengthB


In [15]:
winner_match

[1, 0, 0, 0]

save csv file of process data

In [15]:
ndff.to_csv("new_test_model.csv", mode="a",header=True)

In [16]:
dff=DataFrame()
dff["name"]=player_name
dff["played"]=played
dff["run_bat"]=run_bat
dff["50s"]=f_50s
dff["100s"]=h_100s
dff["batted"]=batted
dff["out"]=out
dff["bowled"]=bowled
dff["wkt_tacken"]=wkt_tacken
dff["5_wkt_taken"]=wkt_5
dff["run_given_bowler"]=run_given
dff["econ"]=econ
dff["over"]=over

raw file having over all career score of individual player of certain years

In [17]:
dff.to_csv("new_final_withTest_with_player.csv", mode="a",header=True)